In [40]:
from sklearn.feature_selection import SelectFdr, f_classif
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import wilcoxon
from scipy.stats import ranksums
from statsmodels.stats.multitest import fdrcorrection
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif,f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, mean_squared_error, precision_score, jaccard_score, fowlkes_mallows_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from boruta import BorutaPy
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold

In [45]:
rna = pd.read_csv('D:/multiomics/2025june/rna_rb.csv', sep=',')
lipid = pd.read_csv('D:/multiomics/2025june/lipid_rb.csv', sep=',')
meta = pd.read_csv('D:/multiomics/2025june/meta_rb.csv', sep=',')

In [46]:
sets_to_intersect = [
    set(rna['id']),
    set(lipid['id']),
    set(meta['id'])
]

common_ids = set.intersection(*sets_to_intersect)


sorted_common_ids = sorted(common_ids)


meta = meta[meta['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()
lipid = lipid[lipid['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()
rna = rna[rna['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()

In [47]:
sum(rna['id']==meta['id']),sum(rna['id']==lipid['id'])

(188, 188)

In [48]:
def get_quantile(X_train, column_name,new_column_name,a,b,position):

    percentile_a = X_train[column_name].quantile(a)
    percentile_b = X_train[column_name].quantile(b)

    X_train[new_column_name] = 0
    X_train.loc[X_train[column_name] <= percentile_a, new_column_name] = 1

    column_to_move = new_column_name
    target_column = position

    column = X_train.pop(column_to_move)
    X_train.insert(target_column, column_to_move, column)
    
    return X_train

In [49]:
rna = get_quantile(rna, 'sVNT_M1_WT_RBD_10_times_dilution','sVNT_M1_WT_RBD',0.25,0,10)
meta = meta.reset_index(drop=True)
lipid = lipid.reset_index(drop=True)
rna = rna.reset_index(drop=True)

In [50]:
f_rna = rna.iloc[:,12:]
f_lipid = lipid.iloc[:,6:]
f_meta = meta.iloc[:,6:]
f_agender = rna.iloc[:,1:3]
label = rna.iloc[:,10]

In [51]:
label

0      0
1      0
2      0
3      0
4      0
      ..
183    1
184    1
185    0
186    1
187    1
Name: sVNT_M1_WT_RBD, Length: 188, dtype: int64

In [52]:
rna['sVNT_M1_WT_RBD_10_times_dilution']

0      43.524180
1      73.985548
2      64.591440
3      79.988883
4      97.832129
         ...    
183     9.912732
184     2.955794
185    49.742672
186    20.428982
187    25.032697
Name: sVNT_M1_WT_RBD_10_times_dilution, Length: 188, dtype: float64

In [53]:
cross_table = pd.crosstab(rna['sVNT_M1_WT_RBD'], rna['Batch'])

In [54]:
cross_table

Batch,Batch01,Batch02
sVNT_M1_WT_RBD,,
0,57,84
1,34,13


In [55]:
combine = pd.concat([f_rna, f_lipid, f_meta], axis=1) 
combine = pd.concat([f_agender, combine], axis=1)

In [56]:
bl = (
    rna['Batch'].astype(str).str.cat(rna['sVNT_M1_WT_RBD'].astype(str), sep='_')
)

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(combine, label, test_size=0.2,stratify=bl,random_state=42)

In [58]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape,bl.shape

((150, 44946), (38, 44946), (150,), (38,), (188,))

In [59]:
bl[X_train.index]

118    Batch01_0
132    Batch01_0
177    Batch01_0
20     Batch02_0
98     Batch02_0
         ...    
160    Batch01_1
136    Batch02_0
163    Batch02_0
13     Batch02_0
175    Batch01_1
Name: Batch, Length: 150, dtype: object

In [60]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from functools import partial
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import f_classif
from joblib import Parallel, delayed
import pandas as pd

def process_fold(train_idx, val_idx, i, X_train, y_train):
    print(f"Processing fold {i}")
    

    X_fold_train, y_fold_train = X_train.iloc[train_idx], y_train.iloc[train_idx]
    

    F_values, p_values = f_classif(X_fold_train, y_fold_train)
    significant_features_mask = p_values < 0.05
    significant_features = X_fold_train.columns[significant_features_mask]
    print(f"Fold {i+1}: {len(significant_features)} significant features")
    X_fold_train_sig = X_fold_train[significant_features]

    estimator = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    boruta_selector = BorutaPy(
        estimator=estimator,
        n_estimators='auto',  
        verbose=0,           
        random_state=42,
        max_iter=100,       
        perc=95,alpha=0.05              
    )
    

    boruta_selector.fit(X_fold_train_sig.values, y_fold_train.values)
    

    selected_features_mask = boruta_selector.support_
    selected_features = significant_features[selected_features_mask]
    

    estimator.fit(X_fold_train_sig[selected_features], y_fold_train)
    importances = estimator.feature_importances_
    

    return [
        {
            'fold': i,
            'feature': feature,
            'importance': importance,
        }
        for feature, importance in zip(selected_features, importances)
    ]


def main():
  
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(partial(process_fold, X_train=X_train, y_train=y_train))(train_idx, val_idx, i)
        for i, (train_idx, val_idx) in enumerate(cv.split(X_train, bl[X_train.index]))
    )
    

    results = [item for sublist in results for item in sublist]
    

    results_df = pd.DataFrame(results)
    

    results_df = results_df.sort_values(['fold', 'importance'], ascending=[True, False])

    print(results_df.head())
    return results_df

if __name__ == '__main__':
    all_feature_counts = main()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  1.5min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  1.5min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.6min remaining:   40.0s


     fold                                    feature  importance
155     0                 C10784__Cannabidiolic.acid    0.024330
48      0                  ENSG00000203497.3__NA__NA    0.020133
28      0  ENSG00000143845.15__ETNK2__protein.coding    0.012031
21      0  ENSG00000125787.11__GNRH2__protein.coding    0.011810
97      0                  ENSG00000250069.1__NA__NA    0.010955


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished


In [61]:
feature_counts = all_feature_counts['feature'].value_counts()
common_features = feature_counts[feature_counts >= 5].index
df_filtered = all_feature_counts[all_feature_counts['feature'].isin(common_features)]

stats = df_filtered.groupby('feature')['importance'].agg(['sum', 'mean', 'count'])
stats = stats.sort_values('mean', ascending=False)

top_feature = stats.index[:13]
feature_selected = list(top_feature)

In [65]:
feature_selected

['ENSG00000203497.3__NA__NA',
 'C10784__Cannabidiolic.acid',
 'ENSG00000258017.3__LOC105369760__ncRNA',
 'X5283130__12.Epileukotriene.B4',
 'C12859__Estradiol.valerate',
 'HMDB0015684__Diloxanide.furoate',
 'HMDB0002237__3.4.Dimethylbenzoic.acid',
 'ENSG00000226668.5__NA__NA',
 'ENSG00000223350.2__IGLV9.49__other',
 'ENSG00000104960.15__PTOV1__protein.coding',
 'ENSG00000115423.19__DNAH6__protein.coding',
 'ENSG00000250069.1__NA__NA',
 'C10935__DEET']

In [87]:
# Prepare training and test sets
Xtrain = X_train[['Age', 'Gender'] + feature_selected]
Xtest = X_test[['Age', 'Gender'] + feature_selected]

# Initialize and train Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(Xtrain, y_train)

# Predict probabilities for the positive class
y_pred_proba = rf.predict_proba(Xtest)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)

print(f'{auc:.2f}')

0.80
